In [ ]:
from zipfile import ZipFile
with ZipFile("main_dataset_folder.zip", "r") as zip_ref:
  zip_ref.extractall("./")
print("Extraction of zip file complete")

In [ ]:
from datasets import  load_dataset, Audio

dataset = load_dataset(
    "csv",
    data_files="/content/main_dataset_folder/main_dataset.csv",
    split = "train"
)

dataset

In [ ]:
dataset = dataset.remove_columns(["Unnamed: 0", "id", "filename", "start", "end", "duration"])

In [ ]:
dataset

In [ ]:
import os

AUDIO_ROOT = "/main_dataset_folder"

def fix_path(example):
  example["file_path"] = os.path.join(AUDIO_ROOT, example["file_path"])
  return example

dataset = dataset.map(fix_path)

In [ ]:
import numpy as np
import librosa
import soundfile as sf
import os

def load_audio_with_librosa(example):
    try:
        audio_array, sample_rate = librosa.load(
            example["file_path"],
            sr=16000,  # 16kHz resample
            mono=True,
            dtype=np.float32
        )

        audio_array = audio_array.astype(np.float32)

        example["audio"] = {
            "array": audio_array,
            "sampling_rate": 16000
        }

        return example
    except Exception as e:
        print(f"Error loading audio file {example['file_path']}: {e}")
    
        example["audio"] = {
            "array": np.zeros(16000, dtype=np.float32),
            "sampling_rate": 16000
        }
        return example

dataset = dataset.map(load_audio_with_librosa)


In [ ]:
dataset["file_path"][0]

In [ ]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

In [ ]:
tokenizer = processor.tokenizer

In [ ]:
def extract_all_chars(batch):
  texts = [str(text) for text in batch["text"] if text is not None]
  all_text = " ".join(texts)
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all text": [all_text]}

vocabs = dataset.map(
    extract_all_chars,
    batched = True,
    batch_size = -1,
    keep_in_memory = True,
    remove_columns = dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

In [ ]:
dataset_vocab- tokenizer_vocab

In [ ]:
dataset["text"][5696]

In [ ]:
import re

def normalize_text(text):

  if text is None or not str(text).strip():
    return ""

  text = str(text).lower()

  text = re.sub(r'[^\w\s\']', '', text)
  text = ' '.join(text.split())

  return text

def add_normalized_text(example):
  example["normalized_text"] = normalize_text(example["text"])

  return example

dataset = dataset.map(add_normalized_text)

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

In [ ]:
dataset_vocab - tokenizer_vocab

In [ ]:
replacements = [
    ("â", "a"),
    ("ç", "ch"),
    ("ğ", "gh"),
    ("ı", "i"),
    ("î", "i"),
    ("ö", "oe"),
    ("ş", "sh"),
    ("ü", "ue"),
    ("û", "u"),
]

def cleanup_text(inputs):
  for src, dst in replacements:
    inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
  return inputs

dataset = dataset.map(cleanup_text)

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)

def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

In [ ]:
dataset

In [ ]:
def prepare_dataset(example):
    audio_data = example["audio"]

    processed_example = processor(
        text=example["normalized_text"],
        audio_target=audio_data["array"],
        sampling_rate=audio_data["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    processed_example["labels"] = processed_example["labels"][0]

    # use SpeechBrain to obtain x-vector
    processed_example["speaker_embeddings"] = create_speaker_embedding(audio_data["array"])

    return processed_example

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

In [ ]:
dataset

In [ ]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)